In [8]:
# Allow packages from venv
import sys
sys.path.append('../venv/lib/python3.7/site-packages')

In [9]:
# Import packages
import pandas as pd

../venv/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [20]:
# Declare filepaths
input_file = "../data/sharktank-db-201810.csv"
output_file = "../results/top1000_channels_oct2018.csv"

We want to get posts that are shares, and not original posts. We do this by getting only those with a `linkWebsite` entry

In [30]:
df = pd.read_csv(input_file)
df2 = df[~df['linkWebsite'].isna()]
df2

/Users/dylansalcedo/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,fbentityid,fbid,channel,createdDate,fromProfile,linkWebsite,linkType,linkEntityId
0,384000414,9689093.0,148993958831413_630550270675777,148993958831413,2018-10-16,148993958831413,https://www.facebook.com/sinagyouth/,facebook,15560098.0
1,384017214,8243065.0,169425650085601_735228656838628,169425650085601,2018-10-16,169425650085601,https://www.facebook.com/sinagyouth/,facebook,15560098.0
2,384003475,6582774.0,305671782850793_1974436032641018,305671782850793,2018-10-21,305671782850793,https://www.facebook.com/manilabulletin/,facebook,21863604.0
3,384010622,16633903.0,1672444669707545_2209033279382012,1672444669707545,2018-10-16,1672444669707545,https://www.facebook.com/sinagyouth/,facebook,15560098.0
4,384025963,21237849.0,1677303235815927_2118990331647213,1677303235815927,2018-10-16,1677303235815927,https://www.facebook.com/sinagyouth/,facebook,15560098.0
...,...,...,...,...,...,...,...,...,...
4674791,292943464,1165505.0,2740278009531382_2746477768911406,2740278009531382,2018-10-19,2740278009531382,https://www.facebook.com/superbalitacebu/,facebook,25912978.0
4674792,298799075,1165505.0,2740278009531382_2746505232241993,2740278009531382,2018-10-19,2740278009531382,https://www.facebook.com/pilipinosakuwait/,facebook,340348.0
4674793,142802890,1165505.0,2740278009531382_2746505378908645,2740278009531382,2018-10-19,2740278009531382,https://www.facebook.com/dailymail/,facebook,8631938.0
4674794,197610701,1165505.0,2740278009531382_2746505088908674,2740278009531382,2018-10-19,2740278009531382,https://www.facebook.com/manilabulletin/,facebook,21863604.0


We now process the data to get the top shared entities:

Since `linkWebsite` identifies the source entity, we group the data by the frequency of the items in this column. `linkEntityId` is the id of the entity as it appears in BigQuery, and `linkType` tells us which type of entity it is

We only get the top 1000 entities so we need to arrange the aggregate table by decreasing number of shares

In [33]:
df3 = (
    df2
    .groupby(['linkEntityId','linkWebsite','linkType'],as_index=False)
    .agg(shares=('linkWebsite','count'))
    .sort_values('shares',ascending=False)
    .reset_index(drop=True)
    .rename(columns={'linkType':'channelType'})
    .astype({'linkEntityId': 'int'})
    .head(1000)
)
df3

,linkEntityId,linkWebsite,channelType,shares
0,3930221,https://www.facebook.com/kamicomph/,facebook,97891
1,7772755,https://www.facebook.com/hugotkarinthadzgel1008/,facebook,41343
2,22021916,https://www.facebook.com/probinsyano-ako-proud...,facebook,37498
3,19140867,https://www.facebook.com/imhopelessph/,facebook,31868
4,2253066,https://www.facebook.com/unhappyofficial/,facebook,29919
...,...,...,...,...
995,4235915,https://www.facebook.com/thefilipinonews.press/,facebook,639
996,13119104,https://www.facebook.com/mysteriousworldss/,facebook,639
997,26301791,https://www.facebook.com/unofficialpage.0/,facebook,639
998,28645659,https://www.facebook.com/bombom.gebe,facebook,638


df3 will then be exported as a csv and fed into the next steps of the pipeline